# Organizando dataset com anotações no formato ".json"

In [ ]:
#Importando as bibliotecas que serão utilizadas no código

import sys
import json
import math
from PIL import Image, ImageDraw
import pickle
import numpy as np
import os

#Import para imprimir barra de carregamento
from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Endereços das pastas:

Todos os arquivos devem estar em uma __pasta raiz__, que contém três pastas, para as imagens e as anotações das imagens de __treino__, __validação__ e __teste__. Cada uma das pastas de __treino__, __validação__ e __teste__ deverá ter duas pastas, para __imagens__ e outra para __anotações__ respectivas.

1. Pasta Raiz
    1. Pasta de Treino
        1. Pasta de Imagens
        2. Pasta de Anotações
    2. Pasta de Validação
        1. Pasta de Imagens
        2. Pasta de Anotações
    3. Pasta de Teste
        1. Pasta de Imagens
        2. Pasta de Anotações
    4. Pasta do dataset organizado
        1. Pasta de Imagens
        2. Pasta de Imagens com as anotações desenhadas
        3. Pasta com arquivos de texto com as coordenadas das anotações
        4. Arquivo .json no formato coco

In [ ]:
#Função que cria uma pasta de nome "folder_name" no diretorio "directory"
def create_folder(directory, folder_name):
    directory = os.path.join(directory, folder_name)
    try:
        os.mkdir(directory)
    except OSError:
        print ("Creation of the folder '%s' failed" % folder_name)
    else:
        print ("Creation of the folder '%s' succeded" % folder_name)

    return directory

In [ ]:
#Função que cria o dicionário com as info de todas as imagens e anotações
def create_dictionary_json(classes):
    dictionary_json = {
        "images" : [],
        "type" : "instances",
        "annotations" : [],
        "categories": []
    }

    for i in range(len(classes)):
        category = {
            'supercategory' : classes[i],
            'id' : i,
            'name' : classes[i]
        }
        dictionary_json['categories'].append(category)

    return dictionary_json

In [ ]:
#Função que salva o dicionário no diretório informado
def save_dictionary_json(directory, file_name, dictionary):
    with open(os.path.join(directory, file_name), 'w', encoding = 'utf-8') as file:
        json.dump(dictionary, file, ensure_ascii = False, indent=4)

In [ ]:
#Função que recebe as coordenadas do ponto e retorna as coordenadas do bbox
def point_to_bbox(x, y, heigth, width, bbox_size):
    heigth_parameter = (heigth-y)/heigth

    final_parameter = bbox_size/heigth_parameter
    parameter_y = 10

    xmin = x-final_parameter
    ymin = y-final_parameter+parameter_y
    xmax = x+final_parameter
    ymax = y+final_parameter+parameter_y


    return xmin,ymin,xmax,ymax

In [ ]:
#Função que cria uma lista com as coordenadas do bounding box, desenha os retangulos nas imagens e atualiza o dicionario
def create_list_bbox(data, classes, heigth, width, bbox_size, draw, dictionary_json, image_id, bbox_id, instances):
    bboxes = []
    for i in range (len(data['shapes'])):
        if (data['shapes'][i]['label'] in classes):
            if (data['shapes'][i]['shape_type'] != 'point'):
                xmin = data['shapes'][i]['points'][0][0]
                ymin = data['shapes'][i]['points'][0][1]
                xmax = data['shapes'][i]['points'][1][0]
                ymax = data['shapes'][i]['points'][1][1]

            else:
                x = data['shapes'][i]['points'][0][0]
                y = data['shapes'][i]['points'][0][1]
                xmin,ymin,xmax,ymax = point_to_bbox(x, y, heigth, width, bbox_size)

            bboxes.append([xmin,ymin,xmax,ymax])
            draw.rectangle([xmin,ymin,xmax,ymax], outline=(0,0,255), width=10)
            #draw.ellipse([xmin,ymin,xmax,ymax], fill=(0,0,255))

            annotation = {
                'area': (xmax-xmin)*(ymax-ymin),
                'iscrowd' : 0,
                'image_id' : image_id,
                'bbox' : [xmin, ymin, xmax-xmin, ymax-ymin],
                'category_id' : classes.index(data['shapes'][i]['label']),
                'id' : bbox_id,
                'ignore' : 0,
                'segmentation' : []
            }
            dictionary_json['annotations'].append(annotation)
            bbox_id += 1
            instances += 1

    return bboxes, draw, dictionary_json, bbox_id, instances

In [ ]:
#Função principal. Cria todos os arquivos em seus diretórios e atualiza o dicionário .json
def create_archives(directory, folder1, directory_folder, format_img, format_ann, classes, bbox_size, directory_img_bbox, directory_img_rgb, directory_ann_txt, archive_name):
    instances = 0

    bar = tqdm(total = len(os.listdir(os.path.join(directory, folder1[0]))))
    dictionary_json = create_dictionary_json(classes)

    image_id = 0
    bbox_id = 0
    for img in os.listdir(os.path.join(directory, folder1[0])):
        ann = os.path.splitext(img)[0]+format_ann

        directory_img = os.path.join(directory, folder1[0], img)
        img_rgb = np.array(Image.open(directory_img))
        heigth, width = img_rgb.shape[:2]
        img_bbox = Image.fromarray(img_rgb)
        draw = ImageDraw.Draw(img_bbox)

        bboxes = []
        if ann in os.listdir(os.path.join(directory, folder1[1])):
            directory_ann = os.path.join(directory, folder1[1], ann)

            with open(directory_ann) as json_file:
                data = json.load(json_file)

            bboxes, draw, dictionary_json, bbox_id, instances = create_list_bbox(data, classes, heigth, width, bbox_size, draw, dictionary_json, image_id, bbox_id, instances)

        new_filename = archive_name+'_'+str(image_id)

        #Salvando arquivos nos diretórios respectivos
        #np.savetxt(os.path.join(directory_ann_txt, new_filename+'.txt'), bboxes, fmt='%d')
        #Image.fromarray(img_rgb).save(os.path.join(directory_img_rgb, new_filename+format_img))
        img_bbox.save(os.path.join(directory_img_bbox, new_filename+format_img))

        image = {
            'file_name' : new_filename+format_img,
            'height' : heigth,
            'width' : width,
            'id' : image_id
        }
        dictionary_json['images'].append(image)

        image_id += 1
        bar.update(1)

    #save_dictionary_json(directory_folder, archive_name, dictionary_json)
    bar.close()
    print('instances = ', instances)

In [ ]:
#Entrada:
#Endereço da pasta raiz
path_root = '/content/drive/MyDrive/Iniciação Científica 2020/dataset_eucalipto'

#Nome da pasta de treino, validação e teste
folder_train = 'frames_Treino'
folder_validation = 'frames_Validacao'
folder_test = 'frames_Teste'
folder0 = [folder_train, folder_validation, folder_test]

#Nome das pastas de imagens e anotações
folder_img = 'imagens'
folder_ann = 'anotacoes'
folder1 = [folder_img, folder_ann]

#Nome das classe(s) utilizada(s) nas anotações
classes = ['tree',]

#Parâmetro para o tamanho da bounding box, caso em que a anotação possui 'shape_type'=='point'.
bbox_size = 15

#formato das imagens e anotações
format_img = '.jpg'
format_ann = '.json'

#Saida:
#Nome da pasta do novo dataset
folder_dataset = 'dataset_'+str(bbox_size)

#Nome das pastas aonde serão salvas as imgns e anotações processadas (Elas serão criadas durante o código)
folder_img_bbox = 'img_bbox' #Pasta das imagens com o bounding box
folder_img_rgb = 'img_rgb' #Pasta das imagens renomeadas
folder_ann_txt = 'ann_txt' #Pasta dos arquivos .txt com as info das anotações
#Nome padrão dos arquivos das pastas mencionadas acima
archive_name = ['train', 'validation', 'test']

#Nome dos arquivos com as anotações, gerados na saída do código
archive_train_ann = 'train_ann' #nome do arquivo .json de treino
archive_validation_ann = 'validation_ann' #nome do arquivo .json de validacao
archive_test_ann = 'test_ann' #nome do arquivo .json de teste


In [ ]:
#Organizando e salvando o dataset na pasta 'folder_dataset'
directory_dataset = create_folder(path_root, folder_dataset)

for iten in range(len(folder0)):
    print("Processing %s set" %(archive_name[iten]))
    directory = os.path.join(path_root, folder0[iten])
    directory_folder = create_folder(directory_dataset, folder0[iten])
    directory_img_bbox = create_folder(directory_folder, folder_img_bbox)
    directory_img_rgb = create_folder(directory_folder, folder_img_rgb)
    directory_ann_txt = create_folder(directory_folder, folder_ann_txt)

    create_archives(directory, folder1, directory_folder, format_img, format_ann, classes, bbox_size, directory_img_bbox, directory_img_rgb, directory_ann_txt, archive_name[iten])

print("Execution terminated.")

Creation of the folder 'dataset_15' failed
Processing train set
Creation of the folder 'frames_Treino' failed
Creation of the folder 'img_bbox' failed
Creation of the folder 'img_rgb' failed
Creation of the folder 'ann_txt' failed


  0%|          | 0/212 [00:00<?, ?it/s]

instances =  3717
Processing validation set
Creation of the folder 'frames_Validacao' failed
Creation of the folder 'img_bbox' failed
Creation of the folder 'img_rgb' failed
Creation of the folder 'ann_txt' failed


  0%|          | 0/70 [00:00<?, ?it/s]

instances =  1646
Processing test set
Creation of the folder 'frames_Teste' failed
Creation of the folder 'img_bbox' failed
Creation of the folder 'img_rgb' failed
Creation of the folder 'ann_txt' failed


  0%|          | 0/71 [00:00<?, ?it/s]

instances =  1326
Execution terminated.


In [ ]:
#Entrada:
#Endereço da pasta raiz
path_root = '/content/drive/MyDrive/Iniciação Científica 2020/dataset_eucalipto2'

#Nome da pasta da imagem nova
folder0 = ["Teste_Eucalipto2"]

#Nome das pastas de imagens e anotações
folder_img = 'imagens'
folder_ann = 'anotacoes'
folder1 = [folder_img, folder_ann]

#Nome das classe(s) utilizada(s) nas anotações
classes = ['tree',]

#Parâmetro para o tamanho da bounding box, caso em que a anotação possui 'shape_type'=='point'.
bbox_size = 15

#formato das imagens e anotações
format_img = '.png'
format_ann = '.json'

#Saida:
#Nome da pasta do novo dataset
folder_dataset = 'dataset_novo_'+str(bbox_size)

#Nome das pastas aonde serão salvas as imgns e anotações processadas (Elas serão criadas durante o código)
folder_img_bbox = 'img_bbox' #Pasta das imagens com o bounding box
folder_img_rgb = 'img_rgb' #Pasta das imagens renomeadas
folder_ann_txt = 'ann_txt' #Pasta dos arquivos .txt com as info das anotações
#Nome padrão dos arquivos das pastas mencionadas acima
archive_name = ['test']

#Nome dos arquivos com as anotações, gerados na saída do código
archive_train_ann = 'train_ann' #nome do arquivo .json de treino
archive_validation_ann = 'validation_ann' #nome do arquivo .json de validacao
archive_test_ann = 'test_ann' #nome do arquivo .json de teste

iten = 0

directory_dataset = create_folder(path_root, folder_dataset)

print("Processing %s set" %(archive_name[iten]))
directory = os.path.join(path_root, folder0[iten])
directory_folder = create_folder(directory_dataset, folder0[iten])
directory_img_bbox = create_folder(directory_folder, folder_img_bbox)
directory_img_rgb = create_folder(directory_folder, folder_img_rgb)
directory_ann_txt = create_folder(directory_folder, folder_ann_txt)

create_archives(directory, folder1, directory_folder, format_img, format_ann, classes, bbox_size, directory_img_bbox, directory_img_rgb, directory_ann_txt, archive_name[iten])


print("Execution terminated.")

Creation of the folder 'dataset_novo_15' failed
Processing test set
Creation of the folder 'Teste_Eucalipto2' failed
Creation of the folder 'img_bbox' failed
Creation of the folder 'img_rgb' failed
Creation of the folder 'ann_txt' failed


  0%|          | 0/101 [00:00<?, ?it/s]

instances =  1148
Execution terminated.


In [ ]:
def print_image(directory):
    %pylab inline
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    img = mpimg.imread(directory)
    imgplot = plt.imshow(img)
    plt.show()